# 0) Multi-layer-Perceptron: MNIST example

Aim: show how to use a mlp in the case of MNIST problem classification

In [17]:
import pandas as pd

train_data = pd.read_csv('Others-Tests/train.csv').values
test_data  = pd.read_csv('Others-Tests/test.csv').values 

In [18]:
print(train_data.shape) # (42000, 785)
print(test_data.shape)  # (28000, 784)

(42000, 785)
(28000, 784)


We are given the train and test data sets. In the train data set, there are 42,000 hand-written images of size 28x28. The first column of the CSV is going to be which digit the image represents(we call this ground truth and/or label), and the rest are 28x28=784 pixels with value ranged in [0, 255]. The test data set contains 28,000 entries and it does not have the ground truth column, because it is our job to figure out what the label actually is.

# Prepare the data with PyTorch

In [19]:
from torch.utils.data import Dataset, DataLoader

In [23]:
class TrainDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, ind):
        x = self.data[ind][1:] / 255.0
        y = self.data[ind][0]
        return x, y
    
class TestDataset(TrainDataset):
 def __getitem__(self, ind):
    x = self.data[ind] / 255.0
    return x
 
train_set = TrainDataset(train_data)
test_set  = TestDataset(test_data)

batch_size = 512
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)

# MODEL

In [24]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear = nn.Linear(28*28, 10)
    def forward(self, x):
        out = self.linear(x)
        return out
    
model = MLP().to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

print(model)

MLP(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


# TRAIN THE MODEL

In [25]:
epochs = 10

model.train()

for epoch in range(epochs):
    losses = []
    for batch_num, input_data in enumerate(train_loader):
        optimizer.zero_grad()
        x, y = input_data
        x = x.to(device).float()
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        
        if batch_num % 40 == 0:
         print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
         print('Epoch %d | Loss %6.2f' % (epoch, sum(losses)/len(losses)))

	Epoch 0 | Batch 0 | Loss   2.31
Epoch 0 | Loss   2.31
	Epoch 0 | Batch 40 | Loss   1.12
Epoch 0 | Loss   1.62
	Epoch 0 | Batch 80 | Loss   0.76
Epoch 0 | Loss   1.27
	Epoch 1 | Batch 0 | Loss   0.77
Epoch 1 | Loss   0.77
	Epoch 1 | Batch 40 | Loss   0.57
Epoch 1 | Loss   0.69
	Epoch 1 | Batch 80 | Loss   0.54
Epoch 1 | Loss   0.63
	Epoch 2 | Batch 0 | Loss   0.50
Epoch 2 | Loss   0.50
	Epoch 2 | Batch 40 | Loss   0.47
Epoch 2 | Loss   0.51
	Epoch 2 | Batch 80 | Loss   0.46
Epoch 2 | Loss   0.50
	Epoch 3 | Batch 0 | Loss   0.48
Epoch 3 | Loss   0.48
	Epoch 3 | Batch 40 | Loss   0.42
Epoch 3 | Loss   0.44
	Epoch 3 | Batch 80 | Loss   0.45
Epoch 3 | Loss   0.43
	Epoch 4 | Batch 0 | Loss   0.44
Epoch 4 | Loss   0.44
	Epoch 4 | Batch 40 | Loss   0.44
Epoch 4 | Loss   0.40
	Epoch 4 | Batch 80 | Loss   0.36
Epoch 4 | Loss   0.39
	Epoch 5 | Batch 0 | Loss   0.39
Epoch 5 | Loss   0.39
	Epoch 5 | Batch 40 | Loss   0.33
Epoch 5 | Loss   0.37
	Epoch 5 | Batch 80 | Loss   0.36
Epoch 5 | Loss   0.3

In [26]:
import csv
model.eval()

with open('mlp_submission.csv', 'w') as f:
    fieldnames = ['ImageId', 'Label']
    writer = csv.DictWriter(f, fieldnames=fieldnames, lineterminator = '\n')
    writer.writeheader()
    image_id = 1

    with torch.no_grad():
        for x in test_loader:
            x = x.to(device).float()

            output = model(x).argmax(dim=1)
            for y in output:
                writer.writerow({fieldnames[0]: image_id,fieldnames[1]: y.item()})
                image_id += 1

In [27]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        out = self.mlp(x)
        #Test branch
        return out

model = MLP().to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
